In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio import SearchIO
import csv
import sqlite3
import time
from multiprocessing import Pool, cpu_count
import sys
from calc_icity_ import *
import ast
from collections import defaultdict
import numpy as np

In [6]:
infile = "tnpB_targetgenes_pfam.filtered.csv"
#infile = "tnpB_targetgenes_pfam.csv"

In [7]:
pfam_df = pd.read_csv(infile).iloc[:,1:]
len(pfam_df['hit_id'].unique())

183

In [8]:
all_query_ids = [pfam for pfam in list(pfam_df['query_id'].sort_values().unique()) if type(pfam) == str]

### look into each target gene, or hit

In [9]:
def get_hits_with_pfam_df(query_ids_suffixes, df):
    all_query_ids = [pfam for pfam in list(pfam_df['query_id'].sort_values().unique()) if type(pfam) == str]
    query_ids = []
    for query_ids_suffix in query_ids_suffixes:
        query_ids += [pfam for pfam in all_query_ids if query_ids_suffix in pfam ]    
    hits_s = df[df["query_id"].isin(query_ids)]['hit_id']
    hits_set = set(hits_s)
    hits_with_pfam_df = df[df["hit_id"].isin(hits_set)]
    print("number filtered hits: " + str(len(hits_with_pfam_df['hit_id'].unique())))
    return hits_with_pfam_df

In [10]:
query_ids_suffixes = ["Acetyl"]
#get_hits_with_pfam_df(query_ids_suffixes, pfam_df)

In [11]:
pd.set_option('display.max_rows', None)
Mer_hits_df = get_hits_with_pfam_df(["Mer"], pfam_df)
np.mean(Mer_hits_df["target_length"])

number filtered hits: 10


169.98245614035088

### analyze pfams in high icity genes near interesting hits

In [12]:
pd.set_option('display.max_rows', 7)
baitlists_hits = [ast.literal_eval(baitlist) for baitlist in Mer_hits_df['baitp100s'].unique()]

In [13]:
allbaits_hits = []
for baitlist in baitlists_hits:
    allbaits_hits += baitlist
allbaits_hits = set(allbaits_hits)

In [14]:
baitlists_all = [ast.literal_eval(baitlist) for baitlist in pfam_df['baitp100s'].unique()]
baitlists_expanded = []
for baitlist in baitlists_all:
    for bait in baitlist:
        if bait in allbaits_hits:
            baitlists_expanded.append(str(baitlist))
            break

In [15]:
len(baitlists_expanded), len(baitlists_hits)

(13, 10)

In [16]:
def get_icityhits_near_pfam_hits_df(query_ids_suffixes, df):
    all_query_ids = [pfam for pfam in list(df['query_id'].sort_values().unique()) if type(pfam) == str]
    query_ids = []
    for query_ids_suffix in query_ids_suffixes:
        query_ids += [pfam for pfam in all_query_ids if query_ids_suffix in pfam]    
    hits_s = df[df["query_id"].isin(query_ids)]['hit_id']
    hits_set = set(hits_s)
    hits_with_pfam_df = df[df["hit_id"].isin(hits_set)]
    
    baitlists_hits = [ast.literal_eval(baitlist) for baitlist in hits_with_pfam_df['baitp100s'].unique()]
    allbaits_hits = []
    for baitlist in baitlists_hits:
        allbaits_hits += baitlist
    allbaits_hits = set(allbaits_hits)
    
    baitlists_all = [ast.literal_eval(baitlist) for baitlist in df['baitp100s'].unique()]
    baitlists_expanded = []
    for baitlist in baitlists_all:
        for bait in baitlist:
            if bait in allbaits_hits:
                baitlists_expanded.append(str(baitlist))
                break
                
    baitlists_set = set(baitlists_expanded)
    icityhits_near_domainhits_df = df[df["baitp100s"].isin(baitlists_set)].dropna()
    
    print("number pfam filtered hits: " + str(len(hits_with_pfam_df['hit_id'].unique())))
    print("number icity hits near pfam filtered hits: " + str(len(icityhits_near_domainhits_df['hit_id'].unique())))
    return icityhits_near_domainhits_df

In [17]:
# still displays original pfam_hits. also not very informative anymore
get_icityhits_near_pfam_hits_df(["Acetyl"], pfam_df).head()

number pfam filtered hits: 1
number icity hits near pfam filtered hits: 5


,query_id,query_accession,query_len,hit_id,hit_evalue,target_length,baitp100s,icity,numer,denom,is_tnpB
16,AAA_14,PF13173.8,130.0,6d7ff89d8cbf492f5d,2.300000e-11,375.0,"['82cbf4e851c4ed4e13', 'cd3e0cf59d8f98c2c0', '...",0.750000,9,12,0
17,AAA_16,PF13191.8,170.0,6d7ff89d8cbf492f5d,1.100000e-07,375.0,"['82cbf4e851c4ed4e13', 'cd3e0cf59d8f98c2c0', '...",0.750000,9,12,0
18,AAA_22,PF13401.8,137.0,6d7ff89d8cbf492f5d,1.600000e-06,375.0,"['82cbf4e851c4ed4e13', 'cd3e0cf59d8f98c2c0', '...",0.750000,9,12,0
19,ATPase_2,PF01637.20,233.0,6d7ff89d8cbf492f5d,8.900000e-22,375.0,"['82cbf4e851c4ed4e13', 'cd3e0cf59d8f98c2c0', '...",0.750000,9,12,0
34,Acetyltransf_1,PF00583.27,117.0,7793fc6506532ffe22,3.500000e-23,202.0,"['82cbf4e851c4ed4e13', '5b04b42d2f75e894da', '...",0.947368,18,19,0


### analyze tnpBs without catalytic domains

In [10]:
inactive_tnpBs = []
with open ("../tnpBs/_inactive_tnpBs.faa") as handle:
    for rec in SeqIO.parse(handle, 'fasta'):
        sequence = rec.seq
        pid = rec.id
        inactive_tnpBs.append(pid)

In [11]:
inactive_tnpBs_set = set(inactive_tnpBs)

In [12]:
sum([hit_id in inactive_tnpBs_set for hit_id in pfam_df["hit_id"]])

0

In [13]:
def extract_inactive_tnpBs(entry, inactive_tnpBs):
    baitp100s_set = set(ast.literal_eval(entry))
    intersection = baitp100s_set.intersection(inactive_tnpBs)
    return intersection

In [14]:
def get_hits_with_inactive_tnpB_bait_df(inactive_tnpBs, df):
#     all_query_ids = [pfam for pfam in list(pfam_df['query_id'].sort_values().unique()) if type(pfam) == str]
#     query_ids = []
#     for query_ids_suffix in query_ids_suffixes:
#         query_ids += [pfam for pfam in all_query_ids if query_ids_suffix in pfam ]    
#     hits_s = df[df["query_id"].isin(query_ids)]['hit_id']
#     hits_set = set(hits_s)
#     hits_with_pfam_df = df[df["hit_id"].isin(hits_set)]
    
    
    baitlists_all = [ast.literal_eval(baitlist) for baitlist in df['baitp100s'].unique()]
    baitlists_expanded = []
    for baitlist in baitlists_all:
        for bait in baitlist:
            if bait in inactive_tnpBs:
                baitlists_expanded.append(str(baitlist))
                break
    hits_with_inactive_tnpB_df = df[df["baitp100s"].isin(baitlists_expanded)].dropna()
    # add column, "inactive tnpBs"
    hits_with_inactive_tnpB_df['inactive tnpBs'] = hits_with_inactive_tnpB_df['baitp100s'].apply(lambda x: extract_inactive_tnpBs(x, inactive_tnpBs))
    tnpB_hits_coloc_inactive_tnpB =  hits_with_inactive_tnpB_df[hits_with_inactive_tnpB_df['is_tnpB'] == 1]
                
    print("number hits with >1 inactive tnpB bait: " + str(len(hits_with_inactive_tnpB_df['hit_id'].unique())))
    print("number tnpB hits with >1 inactive tnpB bait: " + str(len(tnpB_hits_coloc_inactive_tnpB['hit_id'].unique())))
    return hits_with_inactive_tnpB_df

In [15]:
pd.set_option('display.max_rows', None)
targets_inactivebaits_df = get_hits_with_inactive_tnpB_bait_df(inactive_tnpBs, pfam_df)

number hits with >1 inactive tnpB bait: 8
number tnpB hits with >1 inactive tnpB bait: 1


In [16]:
targets_inactivebaits_df["hit_id"].unique()

array(['c7a1b8f01c5b7c48a2', '7205dff302ff900300', '23422406293d40c201',
       '6bf6c4c7da68779d7a', 'c1050b21cc75640d51', 'aa6af7e9289c3558d3',
       '986ad2cde69643b021', 'd2246f26c16fb9eec9'], dtype=object)

In [17]:
# verify the number of inactive tnpB baits that have a nearby target
inactive_baits_near_target = set()
for tnpB_set in list(targets_inactivebaits_df["inactive tnpBs"]):
    inactive_baits_near_target.update(tnpB_set)

In [22]:
set(inactive_baits_near_target), set(inactive_tnpBs)

({'0648051c710ffc3f99',
  '0da5a5d756496ec587',
  '0f08a20cfe029876e3',
  '124625bd293e1e735d',
  '13d95e27e737ac62a9',
  '15ce911abbf167916e',
  '31428f0f03af2a9999',
  '476fa0561841de3b03',
  '58587edff93c9575ac',
  '6be956b3dd0dfba935',
  '70e9d1aa2ac3c57a62',
  'a9a1793552f754a2c3',
  'b9cc7b8a78f75d3bc5',
  'c328360c354617b7a9',
  'ee5c498bf075a0ad33',
  'f6e74b8eea638f8073'},
 {'0648051c710ffc3f99',
  '0da5a5d756496ec587',
  '0f08a20cfe029876e3',
  '124625bd293e1e735d',
  '13d95e27e737ac62a9',
  '15ce911abbf167916e',
  '31428f0f03af2a9999',
  '476fa0561841de3b03',
  '58587edff93c9575ac',
  '6be956b3dd0dfba935',
  '70e9d1aa2ac3c57a62',
  '998f3b369603e3d75b',
  'a9a1793552f754a2c3',
  'b9cc7b8a78f75d3bc5',
  'c328360c354617b7a9',
  'ee5c498bf075a0ad33',
  'f6e74b8eea638f8073'})

In [319]:
targets_inactivebaits_df[targets_inactivebaits_df["hit_id"] == '0648051c710ffc3f99']

,query_id,query_accession,query_len,hit_id,hit_evalue,target_length,baitp100s,icity,numer,denom,is_tnpB,inactive tnpBs
